In [71]:
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("Mnist_data/", one_hot=True)

Extracting Mnist_data/train-images-idx3-ubyte.gz
Extracting Mnist_data/train-labels-idx1-ubyte.gz
Extracting Mnist_data/t10k-images-idx3-ubyte.gz
Extracting Mnist_data/t10k-labels-idx1-ubyte.gz


In [72]:
import pandas

In [73]:
import tensorflow as tf
tf.convert_to_tensor(mnist.train.images).get_shape()

TensorShape([Dimension(55000), Dimension(784)])

In [74]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000029148C558D0>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000029148C55898>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x0000029148C55978>)

In [75]:
len(mnist)

3

In [76]:
len(mnist.train.images)

55000

mnist image는 배열형태의 객체이므로 텐서플로우의 convert_to_tensor를 이용해 먼저 텐서로 변환한 후에 get_shape()

In [77]:
print (type(mnist.train.images))
print (type(tf.convert_to_tensor(mnist.train.images)))

<class 'numpy.ndarray'>
<class 'tensorflow.python.framework.ops.Tensor'>


뉴런이 어떻게 학습하는가?<br>
뉴런은 가중치 W와 offset b를 학습해서 어떻게 점을 분류하는가?<br>
z=b+합(xiWi)<br>
만약 z가 양수라면 출력값 y는 1, 음수라면 출력값 y는 0

뉴런에 레이블 달린 데이터를 공급한 다음 얻은 결과를 실제값과 비교<br>
그 후 오차를 최소화하기 위해 W와 b를 조정

z=f(W*x+b)이고, 이때 F를 활성화함수라고 하고, 시그모이드 함수, tanh 함수, relu 함수가 있다. <br>
출력계층에서 두 가지 이상의 클래스로 데이터를 분류하고 싶을 때는 시그모이드 함수의 일반화된 형태인 소프트맥스 함수를 활성화함수로.

SIGMOID 함수: f(X)=1/(1+e*-X)인데, 이때 X가 큰값일때 1, 작은 값일 때 0<br>
SOFTMAX 함수: 각 클래스에 대한 확률을 얻게 해주어 분류문제에 적합( 클래스의 확률 합은 1이고, 가장 높은 확률을 가진 클래스가 결과값이 됨)

In [78]:
W=tf.Variable(tf.zeros([784,10]))
b=tf.Variable(tf.zeros([10]))

In [79]:
x=tf.placeholder("float",[None, 784])

The first dimension of the placeholder is None, meaning we can have any number of rows.

In [80]:
y=tf.nn.softmax(tf.matmul(x,W)+b)

In [81]:
W=tf.Variable(tf.zeros([784,10]))

In [82]:
b=tf.Variable(tf.zeros([10]))

In [83]:
x=tf.placeholder("float",[None, 784])

In [84]:
y

<tf.Tensor 'Softmax_4:0' shape=(?, 10) dtype=float32>

교차 엔트로피 에러<br>
기대하는 값과 실제 출력값 사이의 오차를 재기 위한 방법은 많다. 평균제곱오차나 유클리드 제곱거리등<br>
신경망에서는 cross entropy error를 이용하는데, 
y는 예측된 확률분포, y'은 레이블이 달린 훈련 데이터로부터 얻은 실제 분포로 -시그마(yi'*log(yi))|

In [85]:
y_=tf.placeholder("float",[None, 10]) #이게 예측된 확률분포가 되겠다.

In [86]:
cross_entropy=-tf.reduce_sum(y_*tf.log(y))

샘플에 대한 오차가 계산되면 다음번 루프반복에서는 비용함수를 줄이기 위해 모델을 수정.
반복적인 최소화과정을 신경망에서는 역전파알고리즘으로 주로 해결<br>
오차를 전방으로 전파하는 것으로 가중치 W를 계산시 출력값으로부터 얻은 오차를 뒤쪽으로 전파, 다층계층신경망에서 중요<br>

교차엔트로피 비용함수와 경사하강법 사용시 매 루프마다 반복마다 오차를 줄이기 위해 매개변수를 얼마만큼 변경해야 할지 계산<br> 즉, learning rate을 실시간으로 결과에 따라서 조정가능 hyperparmeter

역전파는 한 마디로 텐서플로가 모델을 훈련시키기 위해 적절한 비용함수의 기울기를 찾기 위한 최적의 알고리즘이며, 데이터 그래프를 실행한다.

In [87]:
train_step=tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [88]:
sess=tf.Session()

In [89]:
sess.run(tf.global_variables_initializer())

In [106]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y:batch_ys})

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_13' with dtype float
	 [[Node: Placeholder_13 = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'Placeholder_13', defined at:
  File "C:\Users\User\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\User\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\User\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\User\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\User\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\User\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    super(ZMQIOLoop, self).start()
  File "C:\Users\Use

In [104]:
print(sess.run(loss, feed_dict={xs:x_data, ys:y_data}))

NameError: name 'loss' is not defined

In [105]:
type(batch_ys)

numpy.ndarray

In [114]:
correct_prediction=tf.equal(tf.argmax(y,1), tf.argmax(y_, 1))

In [116]:
accuracy=tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [ ]:
print sess.run(accurac7)

In [117]:
x = tf.placeholder("float", [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
matm = tf.matmul(x, W)
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder("float", [None, 10])
cross_entropy = -tf.reduce_sum(y_ * tf.log(y))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

InternalError: Blas GEMM launch failed : a.shape=(100, 784), b.shape=(784, 10), m=100, n=10, k=784
	 [[Node: MatMul_9 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](_recv_Placeholder_16_0/_3, Variable_26/read)]]

Caused by op 'MatMul_9', defined at:
  File "C:\Users\User\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\User\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\User\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\User\Anaconda3\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\User\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\User\Anaconda3\lib\site-packages\zmq\eventloop\ioloop.py", line 177, in start
    sup